In [1]:
import datetime as dt
import os
import netCDF4 as nc
import matplotlib as mpl
import time

import numpy as np
import numpy.ma as ma
from scipy.spatial import distance
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.colors as mpl_colors
from matplotlib import rc
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import gaussian_kde
from mpl_toolkits.mplot3d import Axes3D




from salishsea_tools import geo_tools, viz_tools
% matplotlib inline

#### 1. First, load and select the desired trajectories. 
I have put the selected rows (nday=50) into an array called points which has the same 5 columns as the original traj.txt file

In [ ]:
# direct = "/ocean/gsgarbi/analysis-giorgio/time_series/results2/20160601_20160630_264d/"
# traj=np.loadtxt(direct+"/traj.txt", delimiter = ' ')

# nday=100
# tunit=3600
# ntfic=1
# lmt=1896
# # time as fraction of 1 tcyc:
# ndaytcyc=nday*24*3600/(tunit*ntfic*lmt)

# # find the location of the nearest time in the trajectories to the desired time
# flocs=np.argmin(np.abs(traj[:,4]-ndaytcyc))
# # get the actual time in the traj file
# nearestT=traj[flocs,4]
# # return only the points matching the desired time
# crit=traj[:,4]==nearestT


In [ ]:
T1 = dt.datetime.now()

path = "/ocean/gsgarbi/analysis-giorgio/time_series/results2/2016/"

dirs = os.listdir( path )

points = [[0,0,0,0,0]]

nday=100
tunit=3600
ntfic=1
lmt=1896
# time as fraction of 1 tcyc:
ndaytcyc=nday*24*3600/(tunit*ntfic*lmt)


for file in dirs:
    print (file)
    
    t1 = dt.datetime.now()
    
    traj=np.loadtxt(path+file+"/traj.txt", delimiter = ' ')



    # find the location of the nearest time in the trajectories to the desired time
    flocs=np.argmin(np.abs(traj[:,4]-ndaytcyc))
    # get the actual time in the traj file
    nearestT=traj[flocs,4]
    # return only the points matching the desired time
    crit=traj[:,4]==nearestT
    
    points = np.concatenate((points, traj[crit,:]), axis = 0)
    print ("each: ", np.shape(traj[crit,:]))
    print ("total", np.shape(points))
    
    t2 = dt.datetime.now()
    print ("each: ", t1, t2)
    
T2 = dt.datetime.now()
print ("Total: ", T1, T2)


20160201_20160229_401d


In [ ]:
print (np.shape(points))

#### 2. Next, load and prep model grid variables that will be used to calculate the ocean volume estimate. The end result are a series of 1d arrays called modlon_oc, modlat_oc, modz_oc, and modV_oc, containing the coordinates and volumes associated with model grid cells containing water (not land)

In [ ]:
# set up array of zeros of shape of model domain; also load tmask, model lons and lats
mesh = nc.Dataset("/ocean/gsgarbi/mesh_mask_downbyone2.nc")
tmask=np.copy(mesh.variables['tmask'])
deptht=np.copy(mesh.variables['gdept_1d'][0,:])
depthw=np.copy(mesh.variables['gdepw_1d'][0,:])
e1t=np.copy(mesh.variables['e1t'][0,:,:])
e2t=np.copy(mesh.variables['e2t'][0,:,:])
e3t=np.copy(mesh.variables['e3t_0'][0,:,:,:])
mesh.close()
fb=nc.Dataset('/ocean/eolson/MEOPAR/NEMO-forcing/grid/bathy_downonegrid2.nc')
nav_lon = np.copy(fb.variables['nav_lon'])
nav_lat = np.copy(fb.variables['nav_lat'])
bathy = np.copy(fb.variables['Bathymetry'])
fb.close()

In [ ]:
# create aligned arrays (like meshgrid) of lat, lon, z, volume for all ocean points in model
modlon=np.tile(nav_lon,(np.shape(e3t)[0],1,1))
modlat=np.tile(nav_lat,(np.shape(e3t)[0],1,1))
modz=np.tile(np.reshape(deptht,(-1,1,1)),(1,np.shape(e3t)[1],np.shape(e3t)[2]))
mod_dx=np.tile(e1t,(np.shape(e3t)[0],1,1))
mod_dy=np.tile(e2t,(np.shape(e3t)[0],1,1))
modV=mod_dx*mod_dy*e3t

# select only those points that contain water. This produces 1d arrays.
modlon_oc=modlon[tmask[0,:,:,:]==1]
modlat_oc=modlat[tmask[0,:,:,:]==1]
modz_oc=modz[tmask[0,:,:,:]==1]
modV_oc=modV[tmask[0,:,:,:]==1]

#### 3. Load the  thalweg model indices and convert them from grid indices to lat, lon using nav_lat and nav_lon (which were loaded from the bathymetry file in step 2). 
Reduce the number of points by selecting only every 10th thalweg point. 

Extend the thalweg vertically to create a mesh.

In [ ]:
thw = np.loadtxt(
    '/ocean/gsgarbi/MEOPAR/tools/bathymetry/thalweg_working.txt',
    delimiter=" ", dtype=int)
thw_lat = nav_lat[thw[:,0], thw[:,1]]
thw_lon = nav_lon[thw[:,0], thw[:,1]]
thw_bot = bathy[thw[:,0], thw[:,1]]

In [ ]:
#thw_lat5=[l for i,l in zip(range(0,len(thw_lat)),thw_lat) if i%5==0]
#thw_lon5=[l for i,l in zip(range(0,len(thw_lon)),thw_lon) if i%5==0]
thwsubN=7
thw_lat_sub=[l for i,l in zip(range(0,len(thw_lat)),thw_lat) if i%thwsubN==0]
thw_lon_sub=[l for i,l in zip(range(0,len(thw_lon)),thw_lon) if i%thwsubN==0]
thw_bot_sub=[l for i,l in zip(range(0,len(thw_bot)),thw_bot) if i%thwsubN==0]
print(len(thw_lon_sub))

In [ ]:
plt.plot(thw_lon,thw_lat, 'k.')
plt.plot(thw_lon_sub,thw_lat_sub, 'r.')
#plt.xlim((-123.5,-123.2))
#plt.ylim(48.8,49.2)
print(' number of points in reduced thalweg is: ',len(thw_lon_sub) )
print(' distance between consecutive points is: ',
      np.mean(geo_tools.haversine(thw_lon_sub[:-1],thw_lat_sub[:-1],
                                thw_lon_sub[1:],thw_lat_sub[1:])), 'km')

In [ ]:
thwZ=10.0 # depth interval in thalweg mesh # 5 is too small for model grid
thwzs=np.arange(thwZ/2.0,440.0,thwZ)
# set up 2d mesh:
thmlon=np.tile(thw_lon_sub,(len(thwzs),1))
thmlat=np.tile(thw_lat_sub,(len(thwzs),1))
thmz=np.tile(np.reshape(thwzs,(-1,1)),(1,len(thw_lon_sub)))
thmbot=np.tile(thw_bot_sub,(len(thwzs),1))
thm_isoc=(thmz<thmbot)
print(np.sum(thm_isoc), 'ocean points in thalweg mesh')

#### 4. Plot points in 3d to get an idea of what we are working with

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points[:,1],points[:,2],points[:,3],c=points[:,3],s=5,lw=0.0)
ax.plot(thw_lon_sub,thw_lat_sub,np.zeros(len(thw_lon_sub)))
ax.plot(thw_lon_sub,thw_lat_sub,-50*np.ones(len(thw_lon_sub)))
#ax.plot(thw_lon10,thw_lat10,-1*np.array(thw_bot10))
ax.view_init(30, -60)
ax.set_xlim([-126,-123.5])
ax.set_ylim([49,51])
ax.set_zlim(-300,0);

#### 5. Define functions that will estimate concentration along thalweg mesh

In [ ]:
# gaussian functions for weighting data according to distance from target point
def gweight(dist,dz,sigx,sigz):
    # if len(dist)>1 and len(sig)=1, repeate sig for length of dist
    if isinstance(dist,list): # handle list input
        dist=np.array(dist)
        dz=np.array(dz)
    if isinstance(dist,np.ndarray): # handle list/vector input
        sigx=sigx*np.ones(dist.shape)
        sigz=sigz*np.ones(dist.shape)
    return np.exp(-(dist**2/(2*sigx**2)+dz**2/(2*sigz**2)))

In [ ]:
# function to estimate weighted values at a single point in the thalweg mesh
# calculate weighted N by inputing coords of points each with value of 1
# calculate weighted V by inputing coords of model water points with volumes 
# of water at each point
# concentration = weighted N / weighted V

def weighted(reflon, reflat, refz, datalon, datalat, dataz, dataval, sigx, sigz):
    
    distx=geo_tools.haversine(datalon,datalat,
                              reflon*np.ones(len(datalon)),reflat*np.ones(len(datalon)))
    distz=dataz-refz
    
    ind = np.array((distx**2<9*sigx**2)&(distz**2<9*sigz**2)) # select points within 3 sig of reference point

    weights = gweight(distx[ind],distz[ind],sigx,sigz)
    
    return np.sum(weights)

In [ ]:
# function to calculate concentration estimates along entire thalweg mesh
def trajConcThw(thmlon,thmlat,thmz,thm_isoc,points,
                modlon_oc,modlat_oc,modz_oc,modV_oc,sigx,sigz):
    thmC=np.nan*np.ones(thmlon.shape)
    thmlon=thmlon[thm_isoc]
    thmlat=thmlat[thm_isoc]
    thmz=thmz[thm_isoc]
    thmC0=np.nan*np.ones(thmlon.shape)
    #thmN0=np.nan*np.ones(thmlon.shape)
    thmV0=np.nan*np.ones(thmlon.shape)
    for ii in range(0,len(thmlon)):
        reflon=thmlon[ii]; reflat=thmlat[ii]; refz=thmz[ii]
        wN=weighted(reflon, reflat, refz, points[:,1], points[:,2], -1.0*points[:,3],
                    np.ones(np.size(points[:,1])), sigx, sigz)
        wV=weighted(reflon, reflat, refz, modlon_oc, modlat_oc, modz_oc, 
                    modV_oc, sigx, sigz)
        # if always using the same thalweg grid (eg every 10th thalweg point and vertical resoluation 10 m) and 
        # the same sigx and sigz, wV could be saved and reloaded to save on calculation time
        thmC0[ii]=wN/wV
        #thmN0[ii]=wN
        thmV0[ii]=wV
    #remove measurements that are artificially large due to small sample volume:
    mV=np.median(thmV0)
    #mN=np.median(thmN0)
    iii=thmV0<0.5*mV
    thmC0[iii]=np.nan
    thmC[thm_isoc]=thmC0
    return thmC

In [ ]:
# choose sigx and sigz to be similar to grid: takes less than 20min

t1 = dt.datetime.now()

sigx=3 #km
sigz=12 #m 5 is too small
C=trajConcThw(thmlon,thmlat,thmz,thm_isoc,points,
                modlon_oc,modlat_oc,modz_oc,modV_oc,sigx,sigz)

t2 = dt.datetime.now()

In [ ]:
print (t1, t2)

#6 Plot

In [ ]:
t1 = dt.datetime.now()

fig,ax=plt.subplots(1,1,figsize=(12,3))
pmesh=ax.pcolormesh(geo_tools.distance_along_curve(thmlon[0,:],thmlat[0,:]),thmz[:,0],np.ma.masked_where(np.isnan(C),C))
ax.set_ylim(450,0)
ax.set_xlabel('Distance (km, arbitrary)')
ax.set_ylabel('Depth (m)')
fig.colorbar(pmesh)
ax.set_title('Particle density')

t2 = dt.datetime.now()
print (t1, t2)

In [ ]:
deptht[-5:]-deptht[-6:-1]

7. Save

In [ ]:
fig.savefig("/ocean/gsgarbi/thalweg/2016_allmonths_{}days".format(nday))